In [6]:
!pip install langchain
!pip install langchain-community
!pip install pypdf
!pip install sentence-transformers
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 14.5 MB/s eta 0:00:0000:0100:01


In [34]:
import os
import json
import boto3
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain_community.llms import SagemakerEndpoint
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler
from typing import Dict
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


import warnings
warnings.filterwarnings('ignore')

In [37]:
# loaders = [
#     PyPDFLoader("docs/uber_2021.pdf"),
#     PyPDFLoader("docs/lyft_2021.pdf")
# ]

loaders = [
    PyPDFLoader("docs/MIZUHO_FINANCIAL_GROUP_1.pdf"),
    PyPDFLoader("docs/MIZUHO_FINANCIAL_GROUP_2.pdf")
]

docs = []
for loader in loaders:
    docs.extend(loader.load())
    
print(len(docs))

51


In [38]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)
print("Done")

Done


In [39]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = FAISS.from_documents(documents=splits, embedding=embedding)
print("Done")

Done


In [211]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": prompt, "parameters": model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]


content_handler = ContentHandler()

runtime = boto3.client("runtime.sagemaker")

llm=SagemakerEndpoint(
        endpoint_name="mistral-llm",
        client=runtime,
        model_kwargs={"temperature": 0.5, "max_new_tokens":750},
        content_handler=content_handler
    )
print("Done")

Done


In [233]:

template = """
You are expert in comparing two PDFs. 
Answer the question as detailed as possible from the provided context, make sure to provide all the details.
If the answer is not in provided context just say, "Answer is not available in the context.", please don't provide the wrong answer.

Provide Confidence Score as well.

Context:\n {context}?\n
Question: \nCompare and list out similarities and differences of contracts based on {question}.\n

Confidence Score:
"""

prompt = PromptTemplate(template = template, input_variables = ["context", "question"])

chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = vector_db.as_retriever(search_kwargs={"k":12}),
    return_source_documents=False,
    chain_type = "stuff",
    chain_type_kwargs={"prompt":prompt}
)
print("Done")

Done


In [234]:

question = "record dates"
response = chain({"query":question})
print(response)

{'query': 'record dates', 'result': '\nYou are master in comparing two PDFs. \nAnswer the question as detailed as possible from the provided context, make sure to provide all the details.\nIf the answer is not in provided context just say, "Answer is not available in the context.", please don\'t provide the wrong answer.\n\nProvide Confidence Score as well.\n\nContext:\n SECTION 4.6 Fixing of Record Date .\nWhenever any cash dividend or other cash distribution shall become payable or any distribution other than cash shall be made, or\nwhenever rights shall be issued with respect to the Deposited Securities, or whenever the Depositary shall receive notice of any meeting of\nholders of Shares or other Deposited Securities, or whenever for any reason the Depositary causes a change in the number of Shares that\nare represented by each American Depositary Share, or whenever the Depositary shall find it necessary or convenient, the Depositary shall\nfix a record date (a) for the determinatio

In [245]:
response_string = response.get("result")
# print (response_string)

# are represented by each American Depositary Share, or whenever the Depositary shall find it necessary or convenient, the Depositary shall?
## -------------- Slicing from here to get result ---------------
# Question: 
# Compare and list out similarities and differences of contracts based on record dates.


# Confidence Score:
# 100%

# Answer:
# Both contracts contain provisions regarding record dates for various events related to the American Depositary Shares (ADS) and the underlying Deposited Securities. The similarities include:

# 1. The Depositary has the authority to fix a record date for various events, such as dividend or distribution payments, the issuance of rights, the determination of voting rights, and changes to the number of Shares represented by each ADS.
# 2. The owners of Receipts (holders of ADS) are entitled to receive dividends, distributions, or net proceeds of sales based on the number of ADS they own on the record date.
# 3. The owners are responsible for any fees assessed by the Depositary on or after the record date.
# 4. The provisions regarding record dates are subject to the terms and conditions of the Deposit Agreement.

# The differences are:

# 1. The first contract refers to the record date for the determination of the owners of Shares or other Deposited Securities, while the second contract mentions the record date for the owners of Receipts.
# 2. The first contract states that the owners on the record date are entitled to give instructions for the exercise of voting rights, while the second contract does not explicitly mention this.
# 3. The first contract states that the Receipts must be executed and delivered, and countersigned by a duly authorized officer of the Registrar, while the second contract does not have this requirement.
# 4. The first contract does not mention the termination of the Deposit Agreement, while the second contract includes provisions for termination.

output = "Question:\n"

sub1 = "Question:"
output = output + response_string.rsplit(sub1, 1)[1]

print(output)

Question:
 
Compare and list out similarities and differences of contracts based on record dates.


Confidence Score:
100%

Answer:
Both contracts contain provisions regarding record dates for various events related to the American Depositary Shares (ADS) and the underlying Deposited Securities. The similarities include:

1. The Depositary has the authority to fix a record date for various events, such as dividend or distribution payments, the issuance of rights, the determination of voting rights, and changes to the number of Shares represented by each ADS.
2. The owners of Receipts (holders of ADS) are entitled to receive dividends, distributions, or net proceeds of sales based on the number of ADS they own on the record date.
3. The owners are responsible for any fees assessed by the Depositary on or after the record date.
4. The provisions regarding record dates are subject to the terms and conditions of the Deposit Agreement.

The differences are:

1. The first contract refers to 